In [2]:
import os
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
import sys
sys.path.insert(0, '../')

import cv2
import torch
import numpy as np
from TensorRT_Inference import TRTInference

In [3]:
model = TRTInference('wav2vec2-conformer.trt', verbose=True)
model.warmup({'input': np.random.randn(1, 16000)})

In [4]:
import torchaudio
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained('wav2vec2-conformer')

In [5]:
def clean(annotation):
    if "'" in annotation:
        print(annotation, f'has \' in {annotation}, removing')
        annotation = annotation.split("'")[0] + annotation.split("'")[1][1:]  # Tokenizer includes "'" but TIL dataset does not, remove the S following '
    annotation = ' '.join(annotation.split())  # Remove extra spaces
    return annotation

In [11]:
audio, sr = torchaudio.load('test/audio/evala_09977.wav')
audio = processor(audio, sampling_rate=16000).input_values[0][0]
audio = np.expand_dims(audio, axis=0)
output = model({'input': audio})['output']
clean(processor.batch_decode(np.argmax(output, axis=-1))[0])

'THE BATTERY LEFT ON THIS FHOND IS ZERO PERCENT TO AT THEIR JOBS'